<a href="https://colab.research.google.com/github/seandaza/Python-The-Fundamentals/blob/master/Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Servicios Rest para Intercambio de Datos

# Qué son los servicios Rest

> REST, abrevia REPRESENTATIONAL STATE TRANSFER. Es una interfaz para conectar varios sitemas basados en el protocolo `HTTP`, Sirve para obtener y generar datos y operaciones, retornando datos en formato `JSON` Y `XML` a través de métodos como `GET, POST, PUT y DELETE`. En el caso de un servicio, en el cual nosotros vamos a almacenar nuestra lógica de negocio, y vamos a servir los datos con una serie de recursos `URL`.



# Flask










## flask

> `Flask` es un framework para desarrollo web escrito en Python. Se puede utilizar para diversos tipos de aplicación, entre ellas, desarrollo de APIs. De hecho, es uno de los mejores frameworks de desarrollo para implementar este tipo de soluciones por lo sencillo que resulta y las facilidades que ofrece.

> Existen muchas maneras de implementar un API REST con Flask. Desde usar el framework con lo que ofrece de base, hasta instalar varias extensiones con diferentes configuraciones.

> Entre las características de Flask destacan que tiene un ecosistema desacoplado de componentes, el núcleo es muy pequeño y permite al programador tomar decisiones en cuanto a la arquitectura. Adicionalmente, brinda el potencial de instalar las extensiones o complementos que en realidad vas a utilizar, de acuerdo al tipo de proyecto que desarrolles.

> En este Notebook veremos cómo consumir una API usando python flask en Google Colab a traves de `ngrok` para hacer la IP pública desde nuestro local. 




# Ngrok


> Ngrok nos permite exponer a internet una URL generada dinámicamente, la cual apunta a un servicio web que se está ejecutando en nuestra máquina local. Por ejemplo: si tenemos un servicio web arrancado en http://localhost:8080, ngrok genera dinámicamente una URL del tipo http://xxxxxx.ngrok.io visible en internet, y que apunta directamente a nuestro localhost.







#Webservice con flask

> Nuestro objetivo en esta sección es  crear `werbservices` y hacer que corran en  `google colab` con la ayuda de `flask`. Para ello, veamos como funciona google colab(GC): 

> Cuando se ejecuta cualquier código en GC, este código se está ejecutando en una máquina virtual en mi entorno local `127.0.0.1`. Esta direccion ip no es accesible desde fuera. Para ello, `ngrok` nos ayuda a exponer esta ip y hacerla visible desde fuera en una url pública.

> veamos como implementar esto de manera práctica



In [1]:
# Instalemos dependencias
!pip install flask-ngrok

In [2]:
#Importemos librerias
from flask_ngrok import run_with_ngrok
from flask import Flask, request
from flask import Flask, jsonify

## Ejemplo Web Site con Flask

Suponga que tiene una lista de productos informáticos almacenados en el archivo `products` de formato `.json`. Necesitamos crear un website que me permita consultar los diversos productos y sus características, entonces para ello, creamos diferentes rutas que me permitan navegar hasta ellos y obtenerlos a través de un método llamado `GET`. Veamos: 

El siguiente diccionario contiene un conjunto de datos inherentes a productos informáticos; los cuales son descritos por su `nombre`, `precio` y `cantidad`. Es un diccionario pequeño pero podemos ver claramente un ejemplo de `rutas` en el mismo.

In [ ]:
products = [
    { "name": "laptop", "price": 800, "quantity": 20 },
    { "name": "mouse", "price": 12, "quantity": 120 },
    { "name": "monitor", "price": 1800, "quantity": 34 }
]

###Descripción de las Rutas

> En la primera ruta, cuando se navega a la ruta `'/ping'`, el navegador nos muestra en formato json un mensaje. 

> En la segunda ruta, cuando se navega a la ruta `'/products'`, el navegador nos muestra el conjunto total de porductos que se hayan en el diccionario dado.

> Si estando el la ruta de `/products` se navega a la subruta `/products/laptop` por ejemplo, entonces el navegador nos muestra detalle de ese producto en específico. 

> Prueba navegando sobre los demas productos o sobre un producto inexistente para que veas lo que te muestra el navegador.




> Para ver el detalle de las rutas, ejecuta el siguiente código, ingresa a la url que entrega `ngrok` y navega por las diferentes rutas para que obtengas los datos. La `ruta raiz` es la que muestra por defecto y podemos personalizarla a tal manera de mostrar lo que consideremos conveniente

In [ ]:
app = Flask(__name__)

#Ruta raiz
@app.route("/")
def index ():
  return "<h1>Office Products</h1>\n<p>Here you found office products!</p>\n<u>navigate to 'producs' rout for more details</u>"


#Ruta /ping que muestra un mensaje al visitarla
@app.route('/ping')
def ping ():
      return jsonify({"message": "Pong"})


#Ruta '/products'
@app.route('/products', methods=['GET'])
def getProducts(): 
  return jsonify(products)

#Ruta '/products/XXXXXX'
@app.route('/products/<string:product_name>', methods=['GET'])
def getProduct(product_name):
      productsFound = [product for product in products if product['name'] == product_name]
      if len(productsFound) > 0:     
        return jsonify({"product": productsFound[0]})
      return jsonify({"message": "Product not found"})




#Si quiere correrlo desde su pc (no en colab) elimine la linea siguiente

# Si quiere correrlo en colab y que ngronk disponibilize el sitio en la web entonces
# Necesitamos iniciar ngrok cuando la app este corriendo.
run_with_ngrok(app)

app.run()



---



##Webservice para Consultar y Modificar una Base de Datos

> Nuestro objetivo será ahora exponer las consultas que podamos hacer desde base de datos local, a una url pública a través de `ngrok`.

> Para ello, hay que tener en cuenta en primer lugar, que nuestra base de datos en este caso, es de tipo relacional; es decir, está conformada por una serie de tablas con información desplegada en las mismas. Es por ello, que en el proceso, hacemos las consultas a la base de datos a través de `sqlite3` y luego transformamos los DataFrames en diccionarios para poder de esta manera, construír las rutas a través de sus llaves.







> Para el siguiente ejemplo, descargue la base de datos `hr.db` en la siguiente ejecución de código. Esta base de datos como ya dijimos, consta de una serie de datos de recursos humanos de detrminada empresa.

In [3]:
!wget -O hr.db https://github.com/andresrosso/resources/blob/main/coding/uan_gc_2021/hr.db?raw=true/

--2021-10-15 17:41:26--  https://github.com/andresrosso/resources/blob/main/coding/uan_gc_2021/hr.db?raw=true/
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/andresrosso/resources/raw/main/coding/uan_gc_2021/hr.db [following]
--2021-10-15 17:41:26--  https://github.com/andresrosso/resources/raw/main/coding/uan_gc_2021/hr.db
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andresrosso/resources/main/coding/uan_gc_2021/hr.db [following]
--2021-10-15 17:41:26--  https://raw.githubusercontent.com/andresrosso/resources/main/coding/uan_gc_2021/hr.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|

##Requisitos

> Para poder desarrollar el API de manera correcta deberás instalar los siguientes requisitos.

* Python

* Flask

* SQlite3

* Jsonify

Tenemos todo listo para crear nuestra API RESTful básica en Python con Flask, empecemos con el código, no sin antes recordar los principales métodos:


##Métodos HTTP

> Como REST trabaja con el protocolo HTTP, ofrece 4 opciones para manipulación de datos:

* GET : Para hacer consultas y obtener un recurso

* POST : Para crear un recurso

* PUT: Para modificar un recurso

* DELETE: Para eliminar un recurso

Importemos las Dependencias necesarias

In [4]:
# Importemos Librerías
import sqlite3 as sq
import pandas as pd
from sqlite3 import Error
from flask import Flask, request, jsonify, render_template
import json

Establezcamos Conexion con la base de datos

In [5]:
# Conexion a la base de datos llamada hr.db
conn = sq.connect('hr.db')
cur = conn.cursor()

Para un mejor reconocimento de las tablas que comprenden esta base de datos, listemos las tablas ejecutando el siguiente código:

In [6]:
# Identifiquemos las tablas que contiene esta Base de Datos
tables_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]

# Lista de Tablas
print(tables_list)

[('regions',), ('sqlite_sequence',), ('countries',), ('locations',), ('departments',), ('jobs',), ('employees',), ('dependents',), ('job_grades',)]


Elijamos una de las Tablas mostradas y establezcamos un website de consulta para ella, haciendo primero una conexion a la base de datos, seguido de una solicitud a la misma y que nos muestre en un formato de Dataframe los datos pedidos:

In [7]:
# Conexion a la base de datos llamada hr.db
conn = sq.connect('hr.db')
cur = conn.cursor()

# Definiendo la Consulta a la BD
q1 = ('select * from employees ')

# Convert the SQL query to Pandas data Frame
r1 = pd.read_sql(q1, conn)

r1.head()

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id
0,100,Steven,King,steven.king@sqltutorial.org,515.123.4567,1987-06-17,4,24000.0,NaN,9
1,101,Neena,Kochhar,neena.kochhar@sqltutorial.org,515.123.4568,1989-09-21,5,17000.0,100.0,9
2,102,Lex,De Haan,lex.de haan@sqltutorial.org,515.123.4569,1993-01-13,5,17000.0,100.0,9
3,103,Alexander,Hunold,alexander.hunold@sqltutorial.org,590.423.4567,1990-01-03,9,9000.0,102.0,6
4,104,Bruce,Ernst,bruce.ernst@sqltutorial.org,590.423.4568,1991-05-21,9,6000.0,103.0,6


Convirtamos esta tabla  un diccionario que es lo mas parecido a Json, para la comunicacion de los datos.

In [ ]:
# Convirtiendo el DataFrame 'employees' en Diccionario
result1 = r1.to_dict()

result1

Por otro lado, listemos las columnas del Dataframe 'employees' para identificarlas

In [9]:
#Listando las columnas del DataFrame 'employees'
container1 = list(result1)
container1

['employee_id',
 'first_name',
 'last_name',
 'email',
 'phone_number',
 'hire_date',
 'job_id',
 'salary',
 'manager_id',
 'department_id']

Creemos la API

Creamos 3 rutas: la Raíz, que va a mostrar de hecho un website con la estructura ya "quemada" de la estructura de la taba "employees" (sólo en esta oportunidad). La segunda ruta, muestra un archivo Json del contenido de la Tabla "employees", y la tercera ruta, permite acceder a cualquier columna de la misma tabla anterior.

In [ ]:
#Corremos la aplicacion flask
app = Flask(__name__)


"""Muestra un website para la ruta raiz

Parameters
----------
None

Returns
-------
html 
    que contiene información de la tabla 'employees'
"""
@app.route("/")
def index ():
  return '''
      <h1>Employees Table</h1>
      <p>Principal Route: 'employees'</p>
      <u>Subroutes:</u>
      <ul>
        <li>employee_id</li>
        <li>first_name</li>
        <li>last_name</li>
        <li>email</li>
        <li>phone_number</li>
        <li>hire_date</li>
        <li>job_id</li>
        <li>salary</li>
        <li>manager_id</li>
        <li>department_id</li>
      </ul>
      '''

"""retorna la Ruta de Empleados

Parameters
----------
None

Returns
-------
Json 
"""
@app.route('/employees', methods=['GET'])
def getEmployees(): 
  return jsonify(result1)


"""Muestra la ruta principal y las subrutas

Parameters
----------
string: atributes_employee (nombre de las columnas)

Returns
-------
Json
    diccionario con todas las posibles columnas de la Tabla `employees`
"""
@app.route('/employees/<string:atributes_employee>', methods=['GET'])
def getProduct(atributes_employee):
  if atributes_employee in container1:
    return jsonify(result1[f'{atributes_employee}'])
  return jsonify({"message": f"{atributes_employee} not Found!"})

 
 

# Iniciamos ngrok cuando la app este corriendo
run_with_ngrok(app)
app.run()

Mas adelante veremos como en la ruta raíz podremos construír una tabla html que me permita ver estéticamente una mejor estructura de los datos y que aparezacan embebidos en la misma.



---



Creemos ahora unas rutas de consulta para la Tabla 'departments'. El procedimiento es el mismo, solo que cambiaremos el nombre de algunas variables haciendo uso de indices numericos para preservar el sentido de las mismas.

In [ ]:
#Webservice para Tabla 'departments':

# Conexion a la base de datos llamada hr.db
conn = sq.connect('hr.db')
cur = conn.cursor()


# Definiendo la Consulta a la BD
q2 = ('select * from departments ')

# Convert the SQL query to Pandas data Frame
r2 = pd.read_sql(q2, conn)

# Convirtiendo el DataFrame 'employees' en Diccionario
result2 = r2.to_dict()
result2

Detallemos las columnas de la Tabla "departments"

In [11]:
#Listando las columnas del DataFrame 'departments'
container2 = list(result2)
container2

['department_id', 'department_name', 'location_id']

Comprueba la API con el método `GET` y navega por sus Subrutas:

In [ ]:
#Corremos la aplicacion flask
app = Flask(__name__)




#Ruta Raíz
"""Muestra un diccionario que contiene la tabla "departments"

Parameters
----------
None

Returns
-------
html 
    que contiene información de la tabla 'departments'
"""
@app.route("/")
def index ():
  return jsonify(result2)


#Subrutas de la Tabla Empleados

"""Muestra un diccionario con datos de cada columna la tabla "departments"

Parameters
----------
None

Returns
-------
Json
    que contiene información de cualquier columna de la tabla 'departments'
"""
@app.route('/departments/<string:atributes_employee>', methods=['GET'])
def getProduct(atributes_employee):
  if atributes_employee in container2:
    return jsonify(result2[f'{atributes_employee}'])
  return jsonify({"message": f"{atributes_employee} not Found!"})

 

# Iniciamos ngrok cuando la app este corriendo.
run_with_ngrok(app) 

app.run()



---



Veamos ahora otra foma de organizar las rutas:

Como la base de datos consta de una serie de tablas, y cada tabla consta de una serie de columnas, entonces basaremos el metodo `GET` en solo dos rutas: la principal que trae la tabla, y la secundaria que trae una columna específica de la tabla seleccionada. Para ello, guardemos en variables las posibles consultas a cada una de las tablas o inclusive, guardamos en una variable, una consulta especifica:

In [7]:
# Definiendo la Consulta a la BD para la ruta principal.

query = ('select * from employees')


Creemos ahora una API que incorpore ademas del metodo `GET`, el método `POST`:

Pero antes que nada, factoricemos el codigo de las rutas creando las funciónes `select_all()`, `select_some()` e `insert_employee()`:

Por otro lado, para la rutra raíz, en esta ocasión vamos a mostrar los datoa organizados en un tabla, para dar un formato o estructura estética a los datos. La ruta secundaria está construída en torno a una función que recibe como parámetro el nombre de la columna de la tabla "employees" a la cuál quiere navegar y denotamos en la funcion como `atributes`. Finalmente, manejaremos una ruta que maneja el método `POST`, la cuál nos permitirá registrar un nuevo empleado, e inclusive, podemos verificar que fue insertado en la base de datos de manra inmediata.

In [ ]:
#Corremos la aplicacion flask
app = Flask(__name__)

# **********Funciones***************
def json_schema():
  # Conexion a la base de datos llamada hr.db
  conn = sq.connect('hr.db')
  cur = conn.cursor()
  # Convert the SQL query to Pandas data Frame
  r = pd.read_sql(query, conn)
  # Convirtiendo el DataFrame en Diccionario
  result = r.to_dict()
  json = jsonify(result)
  return json


def select_all():
  # Conexion a la base de datos llamada hr.db
  conn = sq.connect('hr.db')
  cur = conn.cursor()
  # Convert the SQL query to Pandas data Frame
  r = pd.read_sql(query, conn)
  # Convirtiendo el DataFrame en Diccionario
  result = r.to_dict()
  json = jsonify(result)
  

  return f'''<h1>Employees</h1>
          <div>
          <table class="table table-striped table-bordered bg-white table-sm">
            <thead>
              <tr>
                <td>First Name</td>
                <td>Last name</td>
                <td>Email</td>
                <td>Phone Number</td>
                <td>Salary</td>
              </tr>
            </thead>
            <tbody>
                    <tr>
                      <td>{result["first_name"]}</td>
                      <td>{result["last_name"]}</td>
                      <td>{result["email"]}</td>
                      <td>{result["phone_number"]}</td>
                      <td>{result["salary"]}</td>
                    </tr>
              </tbody>
            </table>
            </div>
            '''
            
def select_some(atributes):
  conn = sq.connect('hr.db')
  cur = conn.cursor()
  # Convert the SQL query to Pandas data Frame
  r = pd.read_sql(query, conn)
  # Convirtiendo el DataFrame en Diccionario
  result = r.to_dict()
  container = list(result)
  if atributes in container:
    return jsonify(result[f'{atributes}']), 200
  return jsonify({"message": f"{atributes} not Found!"}),404


def insert_employee():
  conn = sq.connect('hr.db')
  cur = conn.cursor()
  sql = ('''insert or replace into employees 
        (department_id, email, employee_id, first_name, hire_date, job_id, last_name, manager_id, phone_number, salary) 
        values ({0}, '{1}', {2}, '{3}', '{4}', {5}, '{6}', '{7}', {8}, {9})'''.format(request.json['department_id'],
        request.json['email'],request.json['employee_id'],request.json['first_name'],request.json['hire_date'],
        request.json['job_id'],request.json['last_name'],request.json['manager_id'],request.json['phone_number'],request.json['salary']))
        
  cur.execute(sql)
  conn.commit()
  if cur.execute(sql):
    return jsonify({"mensaje": "registrado!"}), 201
  return jsonify({"mensaje": "Error"}), 500


#*******************Rutas************************

#Ruta Raiz
@app.route("/")
def indice():
  return json_schema()


#Ruta que muestra el esquema de la Tabla 

"""Muestra un website para la ruta raiz

Parameters
----------
None

Returns
-------
html 
    que contiene información de la tabla 'employees'
"""
@app.route("/schema")
def index ():
  return select_all()
  

#Rutas Secundarias / Navegando sobre las columnas de la Tabla principal

"""Muestra un diciconario con los datos de cualquier subruta elegida

Parameters
----------
str: atributes
nombre de la subruta

Returns
-------
json
    que contiene información de la ruta secundaria '/<string:atributes>'
"""
@app.route('/<string:atributes>', methods=['GET'])
def setEmployee(atributes):
  return select_some(atributes)



#Ruta Bajo el Método POST

"""Registra un nuevo empleado

Parameters
----------
None

Returns
-------
json
    Muestra un diciconario con el mensaje de confirmacion de nuevo empleado registrado
"""
@app.route('/insert_employee', methods=['POST'])
def register():
  return insert_employee()




  




# Iniciamos ngrok cuando la app este corriendo.
run_with_ngrok(app)

app.run()
